In [150]:
import os
import requests
import time

In [83]:
base_url = "https://api.mangadex.org"
languages = ["en"]

In [132]:
def download_chapter(chapter_id, chapter_number):
    
    r = requests.get(f"{base_url}/chapter/{chapter_id}")
    chapter_title = r.json()["data"]["attributes"]["title"]
    #print(chapter_id) 
    #print(chapter_number)
    #print(chapter_title)
    
    print(f"{base_url}/at-home/server/{chapter_id}")
    r = requests.get(f"{base_url}/at-home/server/{chapter_id}")
    r_json = r.json()
        
    if "errors" in r_json:
        print("For some reason, cannot proccess given chapter.")
        return

    host = r_json["baseUrl"]
    chapter_hash = r_json["chapter"]["hash"]
    data = r_json["chapter"]["data"]
    data_saver = r_json["chapter"]["dataSaver"]
        
    for (page_number, page) in enumerate(data):
        #page_number = int(page[0:page.find("-")])
        page_url = f"{host}/data/{chapter_hash}/{page}"
        print(page_number)
        print(page_url)
        #r = requests.get(f"{host}/data/{chapter_hash}/{page}")
        
        #with open(f"{folder_path}/{page}", mode="wb") as f:
        #    f.write(r.content)

    print(f"Downloaded {len(data)} pages.")

In [133]:
def get_given_value_for_key(manga, key):
    if key in manga:
        return manga[key]
    else:
        return None

In [134]:
def find_given_relationship(manga, relationship_type):
    return next(item for item in manga["relationships"] if item["type"] == relationship_type)

In [146]:
def download_manga(title):
    # Searching for the id of the given manga
    res = requests.get(
        f"{base_url}/manga",
        params={"title": title, "includes[]": ["author", "manga", "cover_art"]}
    )
        
    if not "data" in res.json():
        print("For some reason, data is not in the first get request.")
        return
    
        
    manga_data = res.json()["data"][0]
    #print(manga_data)

    manga_id = manga_data["id"]
    title = manga_data["attributes"]["title"]["en"]
    description = manga_data["attributes"]["description"]["en"]
    status = manga_data["attributes"]["status"]
    public_demographic = manga_data["attributes"]["publicationDemographic"]
    # if (public_demographic == "shounen" or public_demographic == "shoujo"):
    #     public_demographic = True
    # else:
    #     public_demographic = False
    
    tags = manga_data["attributes"]["tags"]
    for tag in tags:
        if tag["attributes"]["group"] == "genre":
            genre_id = tag["id"]
            genre_name = tag["attributes"]["name"]["en"]
    
    
    author_data = find_given_relationship(manga_data, "author")
    author_id = author_data["id"]
    author_name = author_data["attributes"]["name"]
    if "biography" in author_data["attributes"] and "en" in author_data["attributes"]["biography"]:
        author_biography = author_data["attributes"]["biography"]["en"]
    else:
        author_biography = None
    
    cover_art_data = find_given_relationship(manga_data, "cover_art")
    filename = cover_art_data["attributes"]["fileName"]
    cover_art_url = f"https://uploads.mangadex.org/covers/{manga_id}/{filename}"
        
    res = requests.get(
        f"{base_url}/manga/{manga_id}/aggregate",
        params={"translatedLanguage[]": languages},
    )
            
    volumes = res.json()["volumes"]
    # dict (chapter_number -> chapter_id)
    chapters = []
    
    
    for (volume_number, volume_data) in volumes.items():
        print(volume_number)
        print("=====================================================")
        #print(volume_data)
        
        chapters = volume_data["chapters"]
        for (chapter_number, chapter_data) in chapters.items():
            chapter_id = chapter_data["id"]
            download_chapter(chapter_id, chapter_number)
            

In [149]:
download_manga("Vinland saga")

none
https://api.mangadex.org/at-home/server/907e921f-349d-4a10-886f-9f59bbf7cd20
0
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/1-5e9a2f38915dfff0d5326b68af6f7c2b3ead4a2001ddcdd57b68817b2fbc1b48.png
1
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/2-36d87320573b1dd188676054ff1f6a46d3f3d85df3c0582db35aa4b61eee00cc.png
2
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/3-7643e6a1f4b81a0e51958cdf131b3d63aecabf22e48b1647963dc7902c560187.png
3
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/4-05c36651e61c3f25fdb44c15576c23b3f3ba4ad6c22ab3d748c062651380f2d2.png
4
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/5-284f6c49e667508fe2bf7f76e0fe9e71054eef678beb6621c0d66c139ccaa532.png
5
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/6-82d805e0f4013ea7616be2cb007ad6d7e79b9feb0b56295db3e9cd307de54a69.png
6
https://uploads.mangadex.org/data/384a2769c19d57aa3e0c35c5d338d9f9/7-08234bb

0
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/1-ddd8b79b82c846438f9581d1a54d0f5b3c72565c3a8148de5e77be5486d6adb3.png
1
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/2-94504b496f8908b75d98fdd0539023009d25cc67db17d2e1157f5e4ca9c68206.png
2
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/3-64dc94ba31fe0a6fb2de9725297bb2e3bb95132652acf29775b3318e0ed90472.png
3
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/4-f2860e9f245bfce56fddf693aac8fe49d277de5d6b5006d2d7f811a9d0316332.png
4
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/5-76b7d0fdf11b81c7a4ab9374c19dce70c9ff800cbfad4e86737bb9a70dcebb78.png
5
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/6-83037cb855312274b8f4851459614cadf7cd77624fb545afe391d6f28edbc5e7.png
6
https://uploads.mangadex.org/data/ca55498ad85968fab955d2d93e723e50/7-42fb0708e80b10c80cb670c0ca44236f8f9929ead93335d372ac2117fec2c60a.png
7
https://uploads.ma

0
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/1-85a870e3d985dd10f6fbf03f315d1efab208e64e78f49c12dd453bb0e840fc77.png
1
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/2-d32f5676909f5ec6508e9e5a1e2e331b677f15814ef8894e42e27311bc8d2c74.png
2
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/3-0abe30c94d9fda77748aee64cdfccaf8f741ae493fbdf7b71d1b0759bd7605e4.png
3
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/4-4c41c6ea2d30e11675c742a0dd2a85863424f36d4755d5aff1289af40bffba01.png
4
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/5-a8ad152428584116c19ceccbe4d85a4df38280c3e1619b7f79ce4383925fa28f.png
5
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/6-da2b84e1e6a159e2391ed15ea926c5ec0289151e9df51091552d4396662e1838.png
6
https://uploads.mangadex.org/data/6b6ac9d161786d69851c72367fdf3db3/7-1fd7a8ac0bae5c6154f051b36397c676cd9ecac03862092f36bca98d49112feb.png
7
https://uploads.ma

0
https://uploads.mangadex.org/data/491075ceb473995bb46c1985de06f128/1-7fd668db4fc677d35e8628e216b9736d7b7b34937e4c8afc108dcac271df3bdb.png
1
https://uploads.mangadex.org/data/491075ceb473995bb46c1985de06f128/2-78c1b488b0702c4efde8306c056aadf2d69ef199c6ae609582d225c563ef958c.png
2
https://uploads.mangadex.org/data/491075ceb473995bb46c1985de06f128/3-6bc9e508d48bb9fe790d82f54d27609fb8030c93999e9a04602cb814638d06c2.png
3
https://uploads.mangadex.org/data/491075ceb473995bb46c1985de06f128/4-f8388d2778fda16d28d86d9d6a6e2dd1d66e453a46a3cfee5aa1d7c9da89a7d8.png
4
https://uploads.mangadex.org/data/491075ceb473995bb46c1985de06f128/5-a6e72fabbbf6c58699fc05fd2cbef1bf5222dd6757aa167fc91cd2923e9aef4a.png
Downloaded 5 pages.
https://api.mangadex.org/at-home/server/d8748a5e-1bc4-48a0-858a-a0097966bbc8
0
https://uploads.mangadex.org/data/d050c6944d03ab93db803a6fea54276a/1-cd48b59ce9935f2817078a17adcd997d7a9f5701bc5df850a8e560088aee0cfd.png
1
https://uploads.mangadex.org/data/d050c6944d03ab93db803a6fea5

0
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/1-2166d9b534c4f79a8e6882cc95863de0fc99b6287a32f3a9d15c213ca0b0fde3.png
1
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/2-336ad1e21cbbe005fcd6a18d498aff575dedbd014dc50cf1def849ae7469673b.png
2
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/3-a41a997120043e0ebfc5fc3ae9f61ba4324aab137bdab5cb67b18885eeb07af2.png
3
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/4-3a60367e5362df6cbd3eacfe787463e4e3a66e6747b1c6de764f1efe40f63bec.png
4
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/5-2cd54fb79b8826b87cbc5222743fac1ac55962c374b2001a6a1847541abc68c0.png
5
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/6-97896d2a8cb13f38a8e79e5ca78a161a1f6c33e5de7610c638059ea48b509e9b.png
6
https://uploads.mangadex.org/data/267a562d7700a0e12217e6e41501650c/7-794aa79e15affc6a879411c6002521081c6781a3eecd3fe253083bb7daab8dfa.png
7
https://uploads.ma

...

0
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x1-4ab48bb97f153d875b4c24f59a407742e0233e793ae3de9f81c64f9a38b4fba7.jpg
1
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x2-5167996564adb9a3f970edb82d3ba1f7f1fedc0355137f724b59d02ba9b807b6.jpg
2
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x3-26b6d1b37122987c13ab81fc2a1bf1cacc23339dac9b3b9e7fffbaa436b9cc02.jpg
3
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x4-fad346c5c2b7622b2847c37b1e099941094b30afb1bc480fe4db10b94abc12e2.jpg
4
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x5-e61009d8afd5e45bb638fde99367cecf61a42ec1bf9fc31ea92b7037cf391b1d.jpg
5
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x6-98ac603280821d16ff1bdadee02756ec08328833d832ae5f1e7ad2cbca179bbf.jpg
6
https://uploads.mangadex.org/data/65b52f92e47255a97291bfbb7280606f/x7-08e1c2d9cf18af3b6aabb8069f3a287a159005c54c40d1484ab31c16f243d9c7.jpg
7
https://upl

0
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x1-2d24b4e58cefd2fadef576bac69491d5f1d87242bae5e315d9d8b33f936089bb.jpg
1
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x2-2e93de20922d5c5d93add21cf3f25eb0e648844154eb5354db572f88b4b625bd.jpg
2
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x3-aa51a8fd7bcef208aabdc1bf281691927dd18ffe804beab3d8c4d191c5c68527.jpg
3
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x4-fd7a5d42dcac8fe34c8129ea4f3a3f3ccb07ff4f9bf5acfca178358d1e1952c2.jpg
4
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x5-58e2d4bba75f7a32ffe7a52cd5f8f0bbd2f3ca5463dd0e9a3ebca62c361f9cc7.jpg
5
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x6-9799a728353597efaf18bbadb75c05669d876c129c937103ac3c58ba893da0e8.jpg
6
https://uploads.mangadex.org/data/6918c7a284d5026e4d29f9f1ca537e69/x7-b7b3b380ba548ffac5ad2949766ea74c7aafb0bc0bbed798cbce2bbb43115f41.jpg
7
https://upl

0
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x1-fb64e74ba2c3cfb264936d7000897f4db027b028f37610f38da3e7e776ed6b18.jpg
1
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x2-8175e99d1ad5a66930b841df89dc5042bcbaac5aa9268dbf3e04940bb5fae700.jpg
2
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x3-231565e42bc39cff38f9fb6a86052de849698059f3e1775bfa1b641ec9213479.jpg
3
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x4-9f8d19ac4de741156c898c6a0e306629b9e158caab0de6abb1e29e7f2391216b.jpg
4
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x5-10bd76afbee0b1cdd2d2181db54bdde8dbf81253e87538a6118fdbfa6361fd4e.jpg
5
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x6-96e0b0ee97fa6c31dd7ab5d48f8e0ac0923cc160b99915ee4b4483a16a6f3ef4.jpg
6
https://uploads.mangadex.org/data/df48d8ff72a5ee430cae4ef7d612d8ad/x7-d9e737a957268c5b56d9f7554dc2831bcae35714dda121e92bafa1f34b781b1e.jpg
7
https://upl

https://api.mangadex.org/at-home/server/060af584-fcdd-4040-9cc4-4e37a9df43fd
0
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x1-9ee5942b2f3219050edf8cdb08e4392d0cda180d9881156d6509382175766449.jpg
1
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x2-09e0279fad040c4b6e06538388673323778829743ac28f3d73b4fd3ec4488d4f.jpg
2
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x3-0aef061c29f56c6a7ebf850739c25bc49a2a1ca4210121076639cc8a991b6770.jpg
3
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x4-db9259520c2222bbadc7b4f9835f43a5c77559e955b2f3b4dfadf56255ed3c9e.jpg
4
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x5-55564647a9514f2b26822cf64bec2738ca43b240810be3642edf07245225bcba.jpg
5
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x6-47f28221167934b52653ae15e21e1d54b9e7af2d16156f445993b84ef4ec7b3b.jpg
6
https://uploads.mangadex.org/data/755b778933bbd1b9d6909ef32c59f9d4/x7-0b611

For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/14c22828-f039-4ed3-8219-a4064e44fcfe
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/4e82b7d2-f687-44ec-abf5-721c9d209d55
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/2e2d80ee-2ab8-4146-a710-043ff4ff9fae
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/f929b7b3-2e91-422c-b495-3e452fe89ad0
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/1c50efb0-df94-451c-92ae-db446719c15e
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/e881ccb9-9fce-4d69-aa70-3d8a6a939bad
For some reason, cannot proccess given chapter.
3
https://api.mangadex.org/at-home/server/63ebd417-10f4-488f-a1f6-486ccf5e0bf7
For some reason, cannot proccess given chapter.
https://api.mangadex.org/at-home/server/537dac82-0526-4486-932b-2ad8601cae6